In [1]:
import telepot
from telepot.loop import MessageLoop
from telepot.namedtuple import InlineKeyboardMarkup, InlineKeyboardButton, ReplyKeyboardMarkup, KeyboardButton
from telepot.delegate import (per_chat_id, per_application, call, create_open, pave_event_space)
from pprint import pprint
import sys
import time
import response_manager
import pandas as pd
import numpy as np
%run get_results.ipynb

[115355   9418  20922  86895]
[('javax.swing.plaf.basic.BasicDirectoryModel-renameFile(java.io.File-java.io.File)', 'Renames a file in the underlying file system.   ', 0.6291374988057696)]


In [2]:
# Simulate a database to store unread messages
class MessageStore(object):
    def __init__(self):
        self._db = {}
        self._results_db = []
        self._results_ontology_db = []
        self.tag = []
     
    def put_tag(self, t):
        if len(self.tag):
            #del self.tag
            self.tag.append(t)
        else:
            self.tag.append(t)
       
    def pull_tag(self):
        t = self.tag
        return t 
    
    def put_results(self, results):
        if len(self._results_db):
            del self._results_db[0]
        self._results_db.append(results)
            
    def pull_results(self):
        if len(self._results_db):
            res = self._results_db
            return res
        else:
            return []

    def put_results_ontology(self, results):
        if len(self._results_ontology_db):
            del self._results_ontology_db[0]
        self._results_ontology_db.append(results)
            
    def pull_results_ontology(self):
        if len(self._results_ontology_db):
            res1 = self._results_ontology_db
            return res1
        else:
            return []    
        
    def put(self, msg):
        chat_id = msg['chat']['id']

        if chat_id not in self._db:
            self._db[chat_id] = []

        self._db[chat_id].append(msg)

    # Pull all unread messages of a `chat_id`
    def pull(self, chat_id):
        
        if len(self._db):
            messages = self._db[chat_id]
            # sort by date
            messages.sort(key=lambda m: m['date'])
            return messages
        else:
            return []
    
    def delete_mess(self, chat_id):
        if len(self._db):
            del self._db[chat_id]
        # Tells how many unread messages per chat_id
        #return [(k,len(v)) for k,v in self._db.items()]

In [3]:
# Accept commands from owner. Give him unread messages.
class MessageHandler(telepot.helper.ChatHandler):
    def __init__(self, seed_tuple, store, **kwargs):
        super(MessageHandler, self).__init__(seed_tuple, **kwargs)
        self._store = store

    def _read_messages(self, messages):
        for msg in messages:
            # assume all messages are text
            self.sender.sendMessage(msg['text'])

    def on_chat_message(self, msg):
        
        #Flag per non entrare nella iterazione del yes or no quando si sta processando per la prima volta il problema 
        flag = True
        
        #print(msg)
        content_type, chat_type, chat_id = telepot.glance(msg)
        
        if content_type == 'text' :
            name = msg["from"]["first_name"]
            txt = str(msg['text'])
        
        else:
            self.sender.sendMessage("I don't understand")
            return
        #print (txt)
        
        #Con questo metodo creiamo una lista con i messaggi precedenti relativi alla chat con l'utente corrente 
        message_history = self._store.pull(chat_id)
          
        #Se il comando è start mando il messaggio di inzio ed elimino i messaggi precedenti salvati relativi all'utente 
        if '/start' == txt :
            #print(message_history)
            #self.sender.sendMessage('Hello {}! I\'m StackBot. Tell me what is your problem involving Python language, try to be as general as possible'.format(name))
            
            markup = ReplyKeyboardMarkup(keyboard=[['Word2vec', 'Tfidf']])
            self.sender.sendMessage('Hello {}! I\'m StackBot.What technology do you want to use?'.format(name), reply_markup = markup)
            
            self._store.delete_mess(chat_id)
            self._store.put(msg)
            
            if len(message_history) > 0 and txt == '/start':
                self._store.delete_mess(chat_id)
                message_history = self._store.pull(chat_id) 
                self._store.put(msg)
            
            
        elif len(message_history) == 0 and txt != '/start':
            
            markup = ReplyKeyboardMarkup(keyboard=[['/start']])
            
            #markup = InlineKeyboardButton(inline_keyboard=[['Logo','Text']])
            
            
            self.sender.sendMessage('You have to write /start to begin the interaction', 
                                    reply_markup = markup)
            
            
        
        #print("Message History", message_history)
        #print("Len Message History", len(message_history))
        
        #or (message_history[len(message_history)]['text'] and message_history[0]['text'] == '/start')
        
        if (len(message_history) == 1 and message_history[0]['text'] == '/start') :
            #controlliamo che il messaggio appena inviato non sia un altro comando 
            if 'entities' in msg:
                self._store.delete_mess(chat_id)
                self.sender.sendMessage("You can't type this in your request, please restart your interaction")
        
            else:
                self._store.put(msg)
                #scrivi il problema
                self.sender.sendMessage('Tell me what is your problem involving python language with {}, try to be as general as possible'.format(message_history[1]['text']))
                
                
        
        #Se l'unico messaggio salvato è il comando start procediamo con il primo passo del processo
        if (len(message_history) == 3 and message_history[0]['text'] == '/start') :
            
            #controlliamo che il messaggio appena inviato non sia un altro comando 
            if 'entities' in msg:
                self._store.delete_mess(chat_id)
                self.sender.sendMessage("You can't type this in your request, please restart your interaction")
                
            #altrimenti sarà il testo del problema da mandare al w2v
            else:
                # resp = response_manager.gen_response(txt)
                #Chiamo la funzione che mi rende la lista dei tags delle domande con maggiore similarità

                
                if (message_history[1]['text'] == 'Word2vec'):
                    results = w2v_stack_result(txt)
                    results_ontology = w2v_ontology_result(txt)
                    self._store.put_results(results)
                    self._store.put_results_ontology(results_ontology)
                
                if (message_history[1]['text'] == 'Tfidf'):
                    results = tfidf_stack_result(txt)
                    #results_ontology = tfidf_ontology_result(txt)
                    self._store.put_results(results)
                    #self._store.put_results_ontology(results_ontology)
                    
                #print(results_ontology)
                #print(self._store.pull_results_ontology())
                
                #Salviamo nel nostro "db" i risultati del w2v e il messaggio relativo al problema dell'utente
               
                self._store.put(msg)
                
                #Chiedo all'utente se il suo problema ha a che fare con il tag appena estratto 
                #Notare che precedentemente sono stati eliminati i tag corrispondenti a parole già scritte dall'utente 
                #nella sua descrizione del problema
            
                sendControl(self, results, self._store, message_history, 0, results_ontology)
                
                flag = False
    
        
        #Analizzo il caso in cui sono presenti i messaggi relativi a /start e alla descrizione del problema
        #Quindi qui vado a scremare i risultati in base all'input dell'utente e al fatto se il tag a lui proposto 
        #centri o meno col suo problema 
        
        if len(message_history) > 1 and flag:
            res = []
            res_ontology = self._store.pull_results_ontology()
               
            if txt == 'Yes' or txt == 'yes':  
                res = filter_results(self._store,1)
                self._store.put_results(res)
                sendControl(self, res, self._store, message_history, 1, res_ontology)
                
            elif txt == 'No' or txt == 'no':
                res = filter_results(self._store,2)
                self._store.put_results(res)
                sendControl(self, res, self._store, message_history, 2, res_ontology)
                
            elif txt == 'Continue' or txt == 'continue':
                res = self._store.pull_results()
                res[0].pop(0)
                self._store.put_results(res[0])
                sendControl(self, res[0], self._store, message_history, 5, res_ontology)
            
            elif txt == 'I don\'t know' or txt == 'i don\'t know':
                res = filter_results(self._store,3)
                self._store.put_results(res)
                sendControl(self, res, self._store, message_history, 3, res_ontology)
                
            elif txt == 'I\'m fine' or txt == 'i\'m fine':
                sendControl(self, res, self._store, message_history, 4, res_ontology[0])
                
            elif txt == 'Try with Tfidf' or txt == 'try with tfidf':
                
                #calcolo il tfidf con la stringa del problema inserita dall'utente
                results = tfidf_stack_result(message_history[2]['text'])
                #results_ontology = tfidf_ontology_result(message_history[2]['text'])
                
                #Vado a salvare il nuovo results su cui ciclare
                self._store.put_results(results)
                #self._store.put_results_ontology(results_ontology)
                
                #Stampo inizio ciclo dell'iterazione del bot con l'utente
                sendControl(self, results, self._store, message_history, 0, [])
                
            elif txt == 'Try with Word2vec' or txt == 'try with word2vec':    
               
                #calcolo il tfidf con la stringa del problema inserita dall'utente
                results = w2v_stack_result(message_history[2]['text'])
                results_ontology = w2v_ontology_result(message_history[2]['text'])
                
                #Vado a salvare i nuovi results su cui ciclare
                self._store.put_results(results)
                self._store.put_results_ontology(results_ontology)
                
                #Stampo inizio ciclo dell'iterazione del bot con l'utente
                sendControl(self, results, self._store, message_history, 0, res_ontology)
            
            self._store.put(msg)
            
                

In [4]:
def checkLenResults(results):
    
    if not results:
        #Ossia che results è vuoto --> []
        return True
    else:
        #Ossia che results non è vuoto --> [[..], [..]]
        return False

In [6]:
def sendControl(self, results, selfStore, message_history, typeTxt, results_ontology):
    
    markup = ReplyKeyboardMarkup(keyboard=[[KeyboardButton(text='Continue'), 'I\'m fine']])
    markup1 = ReplyKeyboardMarkup(keyboard=[[KeyboardButton(text='/start')]])
    markup2 = ReplyKeyboardMarkup(keyboard=[[ 'Yes' ,KeyboardButton(text='No'), 'I don\'t know']])
    markup3 = ReplyKeyboardMarkup(keyboard=[[KeyboardButton(text='Try with Tfidf')]])
    markup4 = ReplyKeyboardMarkup(keyboard=[[KeyboardButton(text='Try with Word2vec')]])
    #Caso di prima domanda 
    if (typeTxt == 0):
        
        print("RESULTS: ", results)
        
        #Dai risultati prelevo tutte le liste dei tag delle varie domande 
        tags = [el[2] for el in results]
        print("TAGSSS: ", tags)
        #Restituire primo tag della prima domanda di Stack
        self.sender.sendMessage("Your question has something to do with: " + tags[0][0] + "?", reply_markup = markup2)
        self.sender.sendMessage("Write 'Yes' for yes or 'No' for no or 'I don\'t know' if you don\'t know")
    
    #Se l'utente ha cliccato YES
    if (typeTxt == 1):
        
        #Restituire la prima esima domanda di Stack
        self.sender.sendMessage("The solution to your problem is likely to have been discussed on this page: https://stackoverflow.com/questions/" + str(results[0][0]) + "/")
        self.sender.sendMessage("Click 'Continue' if you want continue the search or click 'I\'m fine' if you want to stop", reply_markup = markup)
    
    #Se l'utente ha cliccato NO
    if (typeTxt == 2):
        
        #Se vi sono ancora delle domande
        if (checkLenResults(results) == False):
            
            #Dai risultati prelevo tutte le liste dei tag delle varie domande 
            tags = [el[2] for el in results]
            
            #Restituire nuovo tag della domanda successiva
            self.sender.sendMessage("Your question has something to do with:" + tags[0][0] + "?", reply_markup = markup2)
            self.sender.sendMessage("Write 'Yes' for yes or 'No' for no or 'I dont\'t know' if you have doubts")
        
        #Se non vi sono più domande, vviene detto che non ci sono più domande con quella tecnologia e si chiede se vuole iniziare con l'altra 
        else:
            if (message_history[1]['text'] == 'Word2vec'):
                self.sender.sendMessage("There aren't other soluctions with this technology, you click 'Try with Tfidf' if you want try with Tfidf", reply_markup = markup3)
            else:
                self.sender.sendMessage("There aren't other soluctions with this technology, you click 'Try with Word2vec' if you want try with Word2vec", reply_markup = markup4)
            
                
    #Se l'utente ha cliccato I'DONT KNOW
    if (typeTxt == 3):
            
        #Se vi sono ancora delle domande
        if (checkLenResults(results) == False):
            
            #Dai risultati prelevo tutte le liste dei tag delle varie domande 
            tags = [el[2] for el in results]
            
            #Restuire nuovo tag della stessa domanda
            self.sender.sendMessage("Your question has something to do with:" + tags[0][0] + "?", reply_markup = markup2)
            self.sender.sendMessage("Write 'Yes' for yes or 'No' for no or 'I dont\'t know' if you have doubts")
        
        #Se non vi sono più domande, vviene detto che non ci sono più domande con quella tecnologia e si chiede se vuole iniziare con l'altra 
        else:
            if (message_history[1]['text'] == 'Word2vec'):
                self.sender.sendMessage("There aren't other soluctions with this technology, you click 'Try with Tfidf' if you want try with Tfidf", reply_markup = markup3)
            else:
                self.sender.sendMessage("There aren't other soluctions with this technology, you click 'Try with Word2vec' if you want try with Word2vec", reply_markup = markup4)
            
     
    #Se l'utente ha cliccato I'M FINE
    if (typeTxt == 4):
        self.sender.sendMessage("The possible function solution in Java is:   " + str(results_ontology[0][0]) + "\n\nThe its description is: " + str(results_ontology[0][1]))
        
    #Se l'utente ha cliccato CONTINUE
    if (typeTxt == 5):
        
        #Se vi sono ancora delle domande
        if (checkLenResults(results) == False):
        
            #Dai risultati prelevo tutte le liste dei tag delle varie domande 
            tags = [el[2] for el in results]
        
            #Restituire primo tag della prima domanda di Stack
            self.sender.sendMessage("Your question has something to do with: " + tags[0][0] + "?", reply_markup = markup2)
            self.sender.sendMessage("Write 'Yes' for yes or 'No' for no or 'I don\'t know' if you don\'t know")
        
        #Se non vi sono più domande, vviene detto che non ci sono più domande con quella tecnologia e si chiede se vuole iniziare con l'altra 
        else:
            if (message_history[1]['text'] == 'Word2vec'):
                self.sender.sendMessage("There aren't other soluctions with this technology, you click 'Try with Tfidf' if you want try with Tfidf", reply_markup = markup3)
            else:
                self.sender.sendMessage("There aren't other soluctions with this technology, you click 'Try with Word2vec' if you want try with Word2vec", reply_markup = markup4)
                

In [7]:
def filter_results(store, choice):
        #Carico in variabili i precedenti risultati
        tmp_results = store.pull_results()
        results = tmp_results[0]
        #tmp_tag_target = store.pull_tag()
        #tag_target= tmp_tag_target[len(tmp_tag_target) - 1] 
        
        tags = []
        tmp_results1 = []
        tmp_results2 = []    
        
        #Dai tutte le domande prelevo tutte le liste dei tag
        tags = [el[2] for el in results]

        
        i = 0
        
        
        if len(tags) > 0:
            #for tag in tags:
                #choice == 1 vuol dire che l'utente è interessato al tag
            if choice == 1: #Y
                
                tag_target = tags[0][0]
                
                #Metto in cima le risposte che hanno il tag "tag_target"
                for item in results:
                    if tag_target not in item[2]:
                        tmp_results1.append(item)
                    else:
                        tmp_results2.append(item)
                        
                results.clear()                        
                results = tmp_results2 + tmp_results1
                        
                #Elimino il tag "tag_target" in tutte le potenziali risposte
                
                #print("\n\nresults: ", results)
                
                j = 0
                    
                for item in results:
                    #print("\n\ntags[0][0]: ", tags[0])
                    #print("\n\nitem[2]: ", item[2])
                    if tag_target in item[2]:
                        results[j][2].remove(tag_target)
                    j = j+1   
                        
                #tmp_tag_target = tag_target
                        
                #i = i-1
                #print('DOPO IL POP', results)
                #else:
                 #   i = i+1
                
            if choice == 2: #N
                
                #Metto in cime le ripsoste che non hanno il tag "tag_target"
                
                for item in results:
                    if tags[0][0] not in item[2]:
                        tmp_results1.append(item)
                    else:
                        tmp_results2.append(item)
                
                results.clear()
                results = tmp_results1 + tmp_results2
                
                #Elimino il tag "tag_target" in tutte le potenziali risposte
                
                j = 0
                
                for item in results:
                    if tags[0][0] in item[2]:
                        results[j][2].remove(tags[0][0])
                    j = j+1    
                
                #Elimino tutte le domande che non hanno tag, ossia []
                
                tmp = []
                for item in results:
                    if item[2]:
                        tmp.append(item)
                results.clear()
                results = tmp
                
            if choice == 3: #I don\'t know
                #ELIMINO QUEL TAG DA SOLO QUELLA DOMANDA
                results[0][2].remove(tags[0][0])
                
                #Elimino tutte le domande che non hanno tag, ossia []
                
                tmp = []
                for item in results:
                    if item[2]:
                        tmp.append(item)
                results.clear()
                results = tmp

        
        return results

In [8]:
results = [ ['ciao', 'wee', ['hgf'] ] , ['ciao', 'whige', [] ], ['wee', 'ci so jpsoj', [] ] ]

tmp = []
for item in results:    
    if item[2]:
        tmp.append(item)

results.clear()        
results = tmp


In [9]:
res = w2v_ontology_result('sdas')
print(res[0])

('javax.imageio.ImageReader-isSeekForwardOnly()', 'Returns true if the current input source has been  marked as seek forward only by passing true as the  seek kForward dOnly argument to the  set tInput method.   ', 0.0)
